# **Get data from Kaggle**

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fizarazvi10","key":"aadc4504649e37d6a08c1a82db2f3830"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download smaranjitghose/corn-or-maize-leaf-disease-dataset

 99% 159M/161M [00:02<00:00, 66.7MB/s]
100% 161M/161M [00:02<00:00, 61.5MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('corn-or-maize-leaf-disease-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# **Data Pre-processing**

Check images and their size

In [6]:
from PIL import Image
import os

# Define the path to your dataset directory
dataset_dir = "/content/data"

# Initialize a dictionary to store image sizes
image_sizes = {}

# Loop through each class directory in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Loop through each image file in the class directory
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            if os.path.isfile(file_path):
                # Open the image and get its size
                with Image.open(file_path) as img:
                    width, height = img.size
                    # Store the image size in the dictionary
                    if (width, height) not in image_sizes:
                        image_sizes[(width, height)] = 1
                    else:
                        image_sizes[(width, height)] += 1

# Print the image sizes and their frequencies
for size, count in image_sizes.items():
    print(f"Image size: {size}, Count: {count}")


Image size: (256, 256), Count: 3852
Image size: (641, 482), Count: 1
Image size: (1024, 577), Count: 1
Image size: (780, 511), Count: 1
Image size: (1200, 1600), Count: 5
Image size: (1936, 1296), Count: 1
Image size: (500, 500), Count: 1
Image size: (1500, 2000), Count: 1
Image size: (300, 200), Count: 3
Image size: (628, 472), Count: 1
Image size: (1000, 585), Count: 1
Image size: (1819, 2731), Count: 1
Image size: (340, 255), Count: 1
Image size: (281, 330), Count: 1
Image size: (1080, 421), Count: 1
Image size: (891, 573), Count: 1
Image size: (400, 350), Count: 1
Image size: (618, 463), Count: 1
Image size: (4608, 3456), Count: 3
Image size: (590, 200), Count: 1
Image size: (400, 267), Count: 2
Image size: (800, 776), Count: 1
Image size: (335, 442), Count: 1
Image size: (1024, 512), Count: 2
Image size: (650, 1024), Count: 1
Image size: (325, 209), Count: 1
Image size: (675, 1200), Count: 7
Image size: (194, 350), Count: 1
Image size: (884, 3071), Count: 1
Image size: (2048, 1536

Rename files

In [7]:
import os

# Define the path to your dataset directory
dataset_dir = "/content/data"

# Loop through each class directory in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Loop through each image file in the class directory
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            if os.path.isfile(file_path):
                # Rename image file to be JPG
                new_file_path = os.path.join(class_dir, file_name.split('.')[0] + '.jpg')
                os.rename(file_path, new_file_path)


Remove images with brightness

In [8]:
import os
from PIL import Image

# Define the path to your dataset directory
dataset_dir = "/content/data"

# Define brightness threshold
brightness_threshold = 200

# Initialize a dictionary to store the count of removed images for each folder
removed_counts = {}

# Loop through each class directory in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        removed_counts[class_name] = 0
        # Loop through each image file in the class directory
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            if os.path.isfile(file_path):
                try:
                    # Open the image
                    with Image.open(file_path) as img:
                        # Calculate brightness
                        grayscale_img = img.convert("L")
                        brightness = sum(grayscale_img.getdata()) / (grayscale_img.size[0] * grayscale_img.size[1])
                        # Check brightness threshold
                        if brightness > brightness_threshold:
                            os.remove(file_path)
                            removed_counts[class_name] += 1
                except (UnidentifiedImageError, OSError):
                    print(f"Skipping {file_path} due to UnidentifiedImageError or OSError")
                    os.remove(file_path)

# Print the count of removed images for each folder
for class_name, count in removed_counts.items():
    print(f"Removed {count} bright images from folder {class_name}")


Removed 0 bright images from folder Blight
Removed 8 bright images from folder Healthy
Removed 0 bright images from folder Common_Rust
Removed 0 bright images from folder Gray_Leaf_Spot


Resize and transform image

In [9]:
import os
from PIL import Image
from PIL import UnidentifiedImageError  # Import UnidentifiedImageError
import torchvision.transforms as transforms

# Define the path to your dataset directory
dataset_dir = "/content/data"

# Define the transformations to be applied
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.Resize((256, 256)),  # Resize images to 256x256
])

# Loop through each class directory in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Loop through each image file in the class directory
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            if os.path.isfile(file_path):
                try:
                    # Open the image
                    with Image.open(file_path) as img:
                        # Apply transformations and save the image
                        transformed_img = transform(img)
                        transformed_img.save(file_path)
                except (UnidentifiedImageError, OSError):
                    print(f"Skipping {file_path} due to UnidentifiedImageError or OSError")
                    os.remove(file_path)


Skipping /content/data/Common_Rust/Corn_Common_Rust (1275).jpg due to UnidentifiedImageError or OSError
Skipping /content/data/Common_Rust/Corn_Common_Rust (1289).jpg due to UnidentifiedImageError or OSError
Skipping /content/data/Common_Rust/Corn_Common_Rust (1295).jpg due to UnidentifiedImageError or OSError
Skipping /content/data/Gray_Leaf_Spot/Corn_Gray_Spot (1).jpg due to UnidentifiedImageError or OSError


Check the size of images

In [10]:
from PIL import Image
import os

# Define the path to your dataset directory
dataset_dir = "/content/data"

# Initialize a dictionary to store image sizes
image_sizes = {}

# Loop through each class directory in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Loop through each image file in the class directory
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            if os.path.isfile(file_path):
                # Open the image and get its size
                with Image.open(file_path) as img:
                    width, height = img.size
                    # Store the image size in the dictionary
                    if (width, height) not in image_sizes:
                        image_sizes[(width, height)] = 1
                    else:
                        image_sizes[(width, height)] += 1

# Print the image sizes and their frequencies
for size, count in image_sizes.items():
    print(f"Image size: {size}, Count: {count}")

Image size: (256, 256), Count: 4175


Divide the dataset and normalize it

In [24]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

# Define dataset directory
data_dir = "/content/data"  # Update with your data directory

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load dataset with transforms
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Define split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate split sizes
num_data = len(dataset)
num_train = int(train_ratio * num_data)
num_val = int(val_ratio * num_data)
num_test = num_data - num_train - num_val

# Split dataset indices
indices = list(range(num_data))
np.random.shuffle(indices)
train_indices = indices[:num_train]
val_indices = indices[num_train:num_train+num_val]
test_indices = indices[num_train+num_val:]

# # Split dataset into train and test
# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_val, num_test])

# # Create data loaders for train and test sets
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Define data loaders for each set
train_loader = DataLoader(dataset, batch_size=32, sampler=SubsetRandomSampler(train_indices))
val_loader = DataLoader(dataset, batch_size=32, sampler=SubsetRandomSampler(val_indices))
test_loader = DataLoader(dataset, batch_size=32, sampler=SubsetRandomSampler(test_indices))




# import torch
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader

# # Define dataset directory
# data_dir = "/content/data"  # Update with your data directory

# # Define transforms for preprocessing
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor()
# ])

# # Load dataset with transforms
# dataset = datasets.ImageFolder(data_dir, transform=transform)

# # Define the ratio for train-test split
# train_ratio = 0.8
# test_ratio = 0.2

# # Calculate split sizes
# num_data = len(dataset)
# num_train = int(train_ratio * num_data)
# num_test = num_data - num_train

# # Split dataset into train and test
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

# # Create data loaders for train and test sets
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# **ResNet18**

In [29]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define hyperparameters
learning_rates = [0.001]
num_epochs = 10
optimizers = [optim.SGD]

# Experiment with different hyperparameters
results = []

for lr in learning_rates:
    for optimizer_fn in optimizers:
        # Load ResNet18 model without pretrained weights
        model = models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))  # Adjust output layer for classification

        # Define device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optimizer_fn(model.parameters(), lr=lr)

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

            # Evaluation on validation set after each epoch
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_accuracy = 100 * correct / total
            print(f"Validation Accuracy: {val_accuracy:.2f}")

        # Evaluation on training set
        model.eval()
        train_predictions = []
        train_labels = []
        with torch.no_grad():
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                train_predictions.extend(predicted.cpu().numpy())
                train_labels.extend(labels.cpu().numpy())

        # Convert lists to numpy arrays
        train_predictions = np.array(train_predictions)
        train_labels = np.array(train_labels)

        # Compute evaluation metrics for training set
        train_accuracy = accuracy_score(train_labels, train_predictions)
        train_precision = precision_score(train_labels, train_predictions, average='macro')
        train_recall = recall_score(train_labels, train_predictions, average='macro')
        train_f1 = f1_score(train_labels, train_predictions, average='macro')

        print(f"Training Accuracy: {train_accuracy:.2f}")
        print(f"Training Precision: {train_precision:.2f}")
        print(f"Training Recall: {train_recall:.2f}")
        print(f"Training F1 Score: {train_f1:.2f}")

        # Evaluation on test set
        model.eval()
        test_predictions = []
        test_labels = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                test_predictions.extend(predicted.cpu().numpy())
                test_labels.extend(labels.cpu().numpy())

        # Convert lists to numpy arrays
        test_predictions = np.array(test_predictions)
        test_labels = np.array(test_labels)

        # Compute evaluation metrics for test set
        test_accuracy = accuracy_score(test_labels, test_predictions)
        test_precision = precision_score(test_labels, test_predictions, average='macro')
        test_recall = recall_score(test_labels, test_predictions, average='macro')
        test_f1 = f1_score(test_labels, test_predictions, average='macro')

        print(f"Test Accuracy: {test_accuracy:.2f}")
        print(f"Test Precision: {test_precision:.2f}")
        print(f"Test Recall: {test_recall:.2f}")
        print(f"Test F1 Score: {test_f1:.2f}")

        # Store results
        results.append({
            'learning_rate': lr,
            'optimizer': optimizer_fn.__name__,
            'train_accuracy': train_accuracy,
            'val_accuracy': val_accuracy,
            'test_accuracy': test_accuracy
        })

# # Plot the results
# fig, ax = plt.subplots(figsize=(10, 6))
# for lr in learning_rates:
#     for optimizer_fn in optimizers:
#         #val_accs = [result['val_accuracy'] for result in results if result['learning_rate'] == lr and result['optimizer'] == optimizer_fn.__name__]
#         ax.plot(val_accs, label=f"LR={lr}, {optimizer_fn.__name__} (Validation)")
#         test_accs = [result['test_accuracy'] for result in results if result['learning_rate'] == lr and result['optimizer'] == optimizer_fn.__name__]
#         ax.plot(test_accs, linestyle='--', label=f"LR={lr}, {optimizer_fn.__name__} (Test)")
# ax.set_xlabel('Epoch')
# ax.set_ylabel('Accuracy')
# ax.legend()
# plt.show()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.8639
Validation Accuracy: 78.18
Epoch [2/10], Loss: 0.5838
Validation Accuracy: 81.29
Epoch [3/10], Loss: 0.4769
Validation Accuracy: 82.73
Epoch [4/10], Loss: 0.4287
Validation Accuracy: 82.49
Epoch [5/10], Loss: 0.4042
Validation Accuracy: 83.93
Epoch [6/10], Loss: 0.3801
Validation Accuracy: 82.25
Epoch [7/10], Loss: 0.3719
Validation Accuracy: 85.13
Epoch [8/10], Loss: 0.3411
Validation Accuracy: 84.89
Epoch [9/10], Loss: 0.3426
Validation Accuracy: 85.85
Epoch [10/10], Loss: 0.3239
Validation Accuracy: 86.33
Training Accuracy: 0.84
Training Precision: 0.83
Training Recall: 0.76
Training F1 Score: 0.75
Test Accuracy: 0.84
Test Precision: 0.85
Test Recall: 0.77
Test F1 Score: 0.76


# MobileNetV2

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import models

# Define hyperparameters
learning_rates = [0.001]
num_epochs = 10
optimizers = [optim.SGD]

# Experiment with different hyperparameters
results = []

for lr in learning_rates:
    for optimizer_fn in optimizers:
        # Load MobileNetV2 model without pretrained weights
        model = models.mobilenet_v2(pretrained=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(dataset.classes))  # Adjust output layer for classification

        # Define device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optimizer_fn(model.parameters(), lr=lr)

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

            # Evaluation on validation set after each epoch
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_accuracy = 100 * correct / total
            print(f"Validation Accuracy: {val_accuracy:.2f}")

        # Evaluation on training set
        model.eval()
        train_predictions = []
        train_labels = []
        with torch.no_grad():
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                train_predictions.extend(predicted.cpu().numpy())
                train_labels.extend(labels.cpu().numpy())

        # Convert lists to numpy arrays
        train_predictions = np.array(train_predictions)
        train_labels = np.array(train_labels)

        # Compute evaluation metrics for training set
        train_accuracy = accuracy_score(train_labels, train_predictions)
        train_precision = precision_score(train_labels, train_predictions, average='macro')
        train_recall = recall_score(train_labels, train_predictions, average='macro')
        train_f1 = f1_score(train_labels, train_predictions, average='macro')

        print(f"Training Accuracy: {train_accuracy:.2f}")
        print(f"Training Precision: {train_precision:.2f}")
        print(f"Training Recall: {train_recall:.2f}")
        print(f"Training F1 Score: {train_f1:.2f}")

        # Evaluation on test set
        model.eval()
        test_predictions = []
        test_labels = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                test_predictions.extend(predicted.cpu().numpy())
                test_labels.extend(labels.cpu().numpy())

        # Convert lists to numpy arrays
        test_predictions = np.array(test_predictions)
        test_labels = np.array(test_labels)

        # Compute evaluation metrics for test set
        test_accuracy = accuracy_score(test_labels, test_predictions)
        test_precision = precision_score(test_labels, test_predictions, average='macro')
        test_recall = recall_score(test_labels, test_predictions, average='macro')
        test_f1 = f1_score(test_labels, test_predictions, average='macro')

        print(f"Test Accuracy: {test_accuracy:.2f}")
        print(f"Test Precision: {test_precision:.2f}")
        print(f"Test Recall: {test_recall:.2f}")
        print(f"Test F1 Score: {test_f1:.2f}")

        # Store results
        results.append({
            'learning_rate': lr,
            'optimizer': optimizer_fn.__name__,
            'train_accuracy': train_accuracy,
            'val_accuracy': val_accuracy,
            'test_accuracy': test_accuracy
        })

# # Plot the results
# fig, ax = plt.subplots(figsize=(10, 6))
# for lr in learning_rates:
#     for optimizer_fn in optimizers:
#         val_accs = [result['val_accuracy'] for result in results if result['learning_rate'] == lr and result['optimizer'] == optimizer_fn.__name__]
#         ax.plot(val_accs, label=f"LR={lr}, {optimizer_fn.__name__} (Validation)")
#         test_accs = [result['test_accuracy'] for result in results if result['learning_rate'] == lr and result['optimizer'] == optimizer_fn.__name__]
#         ax.plot(test_accs, linestyle='--', label=f"LR={lr}, {optimizer_fn.__name__} (Test)")
# ax.set_xlabel('Epoch')
# ax.set_ylabel('Accuracy')
# ax.legend()
# plt.show()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/10], Loss: 1.0553
Validation Accuracy: 48.92
Epoch [2/10], Loss: 0.9368
Validation Accuracy: 52.76
Epoch [3/10], Loss: 0.8100
Validation Accuracy: 63.31
Epoch [4/10], Loss: 0.6858
Validation Accuracy: 73.14
Epoch [5/10], Loss: 0.5924
Validation Accuracy: 76.74
Epoch [6/10], Loss: 0.5259
Validation Accuracy: 81.77
Epoch [7/10], Loss: 0.4593
Validation Accuracy: 81.53
Epoch [8/10], Loss: 0.4292
Validation Accuracy: 83.69
Epoch [9/10], Loss: 0.4063
Validation Accuracy: 83.93
Epoch [10/10], Loss: 0.3814
Validation Accuracy: 84.41
Training Accuracy: 0.82
Training Precision: 0.76
Training Recall: 0.72
Training F1 Score: 0.68
Test Accuracy: 0.83
Test Precision: 0.89
Test Recall: 0.73
Test F1 Score: 0.69
